In [1]:
#Importamos pandas
import pandas as pd

In [103]:
#Importamos los datasets que vamos a utilizar de manera auxiliar
google = pd.read_pickle('metadata-sitios/google.pkl')
duplicados_google_df = pd.read_csv('metadata-sitios/duplicados.csv',index_col=0)

In [ ]:
#Filtramos con los gmaps de los restaurants y exportamos todos los datasets de reviews de los estados
estados = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Columbia','Georgia',
           'Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan',
           'Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New_Hampshire','New_Jersey','New_Mexico','New_York',
           'North_Carolina','North_Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode_Island','South_Carolina','South_Dakota',
           'Tennessee', 'Texas', 'Utah','Vermont', 'Virginia', 'Washington', 'West_Virginia', 'Wisconsin', 'Wyoming']
suma = 0
for e in estados:
    for i in range(1,50):
        if i == 1:
            estado = pd.read_json('reviews-estados/review-'+e+'/1.json',lines=True)
            suma+= estado.shape[0]
        else:
            try:
                estado = pd.concat([estado,pd.read_json('reviews-estados/review-'+e+'/'+str(i)+'.json',lines=True)])
                suma+= estado.shape[0]
            except:
                print(i)
                break
    estado_filtrado = estado[estado['gmap_id'].isin(google['gmap_id'].values)]
    estado_filtrado.to_csv('reviews-estados/'+e+'.csv',index=False,sep=';',escapechar='\\')

In [10]:
#Vemos cuantos registros eran originalmente.
suma

660096359

Eran 660.096.359 registros. Se redujeron y guardaron en archivos csv separados por estado.

In [130]:
#Concatenamos todo y vamos a revisar
cant_registros = 0
for e in estados:
    if e == 'Alabama':
        estado = pd.read_csv('reviews-estados/'+e+'.csv',sep=';')
    else:
        estado = pd.concat([estado,pd.read_csv('reviews-estados/'+e+'.csv',sep=';')],ignore_index=True)

In [131]:
estado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11900482 entries, 0 to 11900481
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 726.3+ MB


In [132]:
#Vemos cuantos usuarios distintos hay
estado['user_id'].unique().size

5629446

In [133]:
#Vemos cuantos restaurants distintos hay
estado['gmap_id'].unique().size

127173

Pasaron a ser 11.900.482 registros. Se redujeron considerablemente. Además podemos ver que:

* Hay 5.629.446 usuarios distintos.
* Hay 127.173 restaurants distintos (menos que los 210.953 que hay en el dataset de los negocios de google)

In [95]:
#Vamos a tomar uno de los estados y lo vamos a revisar para limpiarlo. Luego aplicaremos el mismo proceso a los otros estados.
alabama = pd.read_csv("reviews-estados/Alabama.csv", sep=";")
alabama.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.179759e+20,Anthony Roberts,1463443013514,4,"On the higher end of price for pizza, but they...",NaN,NaN,0x8889221157fb3455:0x5c125c40c3eccc2a
1,1.143165e+20,Jonathan Robert,1447623939865,3,"Food was ok, felt like the atmosphere as well ...",NaN,NaN,0x8889221157fb3455:0x5c125c40c3eccc2a
2,1.136326e+20,Brian Harvey,1469293549247,4,"Good food, service so so",NaN,NaN,0x8889221157fb3455:0x5c125c40c3eccc2a
3,1.076840e+20,Ashley Maddox,1442279219480,5,Love it,NaN,NaN,0x8889221157fb3455:0x5c125c40c3eccc2a
4,1.126036e+20,West Martin,1382634896130,5,Yum,NaN,NaN,0x8889221157fb3455:0x5c125c40c3eccc2a


In [96]:
alabama.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173315 entries, 0 to 173314
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  173315 non-null  float64
 1   name     173315 non-null  object 
 2   time     173315 non-null  int64  
 3   rating   173315 non-null  int64  
 4   text     100068 non-null  object 
 5   pics     4462 non-null    object 
 6   resp     18527 non-null   object 
 7   gmap_id  173315 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 10.6+ MB


Para el caso de las reviews debemos tener los siguientes campos:

* Id_usuario
* Id_Restaurant
* Rating
* Reseña
* Timestamp

No vamos a usar las fotos (pics) para ningún análisis; las respuestas del local tampoco. Tampoco nos interesa como se llama el usuario. Todas estas columnas las vamos a eliminar.

In [97]:
#Eliminamos columnas
alabama.drop(columns=['name','pics','resp'],inplace=True)
alabama.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173315 entries, 0 to 173314
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  173315 non-null  float64
 1   time     173315 non-null  int64  
 2   rating   173315 non-null  int64  
 3   text     100068 non-null  object 
 4   gmap_id  173315 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 6.6+ MB


In [98]:
#Cambiamos los nombres de las columnas y reordenamos
alabama.rename(columns={'user_id':'Id_Usuario','time':'Timestamp','text':'Reseña','gmap_id':'Id_Restaurant','rating':'Rating'},inplace=True)
alabama = alabama[['Id_Usuario','Id_Restaurant','Rating','Reseña','Timestamp']]
alabama.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173315 entries, 0 to 173314
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Id_Usuario     173315 non-null  float64
 1   Id_Restaurant  173315 non-null  object 
 2   Rating         173315 non-null  int64  
 3   Reseña         100068 non-null  object 
 4   Timestamp      173315 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 6.6+ MB


In [99]:
#Descartamos duplicados
alabama.drop_duplicates(inplace=True)
alabama.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166894 entries, 0 to 173314
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Id_Usuario     166894 non-null  float64
 1   Id_Restaurant  166894 non-null  object 
 2   Rating         166894 non-null  int64  
 3   Reseña         93861 non-null   object 
 4   Timestamp      166894 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 7.6+ MB


Ahora vamos a utilizar el dataframe de duplicados de google que hayamos en el notebook **"negocios_google.ipynb"** para cambiar el los Ids de los restaurant a los que corresponden.

In [104]:
duplicados_google_df.head()

,Conservar,Cambiar
0,0x89c2f5d10275956b:0x678f175621275540,['0x89c2f556a7d50035:0x1588f291f277f706']
1,0x88d9b76485f1a105:0x92bc2888314d41f4,['0x88d9b1e28cc6f4db:0x4dc90cfb7b4f01b7']
2,0x88ded74449b67443:0x6f5ce881bf9883d0,['0x88ded58b9368604d:0x4fb1d504b6ae7eee']
3,0x89b60170d06531f1:0xcb85dc5e00da9e1f,['0x89b60170d0e68baf:0x67a51b3fda1b9d5e']
4,0x89c259a11e60603d:0x92f338b57ed650bb,['0x89c2597fb6fffff9:0xd6e4a37d0c56f328']


In [105]:
duplicados_google_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3515 entries, 0 to 3514
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Conservar  3515 non-null   object
 1   Cambiar    3515 non-null   object
dtypes: object(2)
memory usage: 82.4+ KB


In [106]:
#Realizamos modificación puesto que se guardó todo como un string largo en vez de lista de strings
for i in range(duplicados_google_df.shape[0]):
    duplicados_google_df['Cambiar'][i] = duplicados_google_df['Cambiar'][i].replace("'",'').strip("[]").replace(' ','').split(',')
duplicados_google_df.head()

,Conservar,Cambiar
0,0x89c2f5d10275956b:0x678f175621275540,[0x89c2f556a7d50035:0x1588f291f277f706]
1,0x88d9b76485f1a105:0x92bc2888314d41f4,[0x88d9b1e28cc6f4db:0x4dc90cfb7b4f01b7]
2,0x88ded74449b67443:0x6f5ce881bf9883d0,[0x88ded58b9368604d:0x4fb1d504b6ae7eee]
3,0x89b60170d06531f1:0xcb85dc5e00da9e1f,[0x89b60170d0e68baf:0x67a51b3fda1b9d5e]
4,0x89c259a11e60603d:0x92f338b57ed650bb,[0x89c2597fb6fffff9:0xd6e4a37d0c56f328]


In [107]:
#Juntamos todos los ids que hay que modificar para ver cuantos hay en el dataframe de alabama
ids_modificar = []
for i in range(duplicados_google_df.shape[0]):
    ids_modificar+=duplicados_google_df['Cambiar'][i]
ids_modificar

['0x89c2f556a7d50035:0x1588f291f277f706',
 '0x88d9b1e28cc6f4db:0x4dc90cfb7b4f01b7',
 '0x88ded58b9368604d:0x4fb1d504b6ae7eee',
 '0x89b60170d0e68baf:0x67a51b3fda1b9d5e',
 '0x89c2597fb6fffff9:0xd6e4a37d0c56f328',
 '0x89d3776954f27fb3:0xb8ad553ba0b644b4',
 '0x883cae87ca5aa68d:0xe917bb36112048a4',
 '0x80dc70e86c8925db:0xe61846655c7013b5',
 '0x80dca1b23fee002b:0xabdd4e58647158c9',
 '0x54950b4309b7bed5:0x4b0abd6b8dc64341',
 '0x86e4d70e0bc33bb3:0x7f92bcb86712a23c',
 '0x80c3532003f6dfc1:0xf8db71a40b6d24bf',
 '0x4cb206b32de485e3:0xe3daa8d9344ccd44',
 '0x89c259e14f18ddb1:0x8427d011aeb768f5',
 '0x88f56bcc00b219e7:0x63ca0343abcaf675',
 '0x883487a1fb2a7d31:0x8c55ba4394796581',
 '0x89c27f40db664e29:0x6486d9d1b08edac5',
 '0x884036f0b452a4fb:0x8cc2a3db14b1a5ab',
 '0x80db511eb3cd9cb9:0x4b2bceb4da60c592',
 '0x8856891c15265a13:0x8066c882c92a4007',
 '0x88f5993cd8a794fd:0x16a3ff3242d99b0a',
 '0x8836fcad41fd1e35:0xeee5fd0cc750067b',
 '0x8800d183397aa209:0x7ac782ee2584db0',
 '0x89aef519214148db:0x68e624cd2267

In [108]:
#Vemos cuantos son
len(ids_modificar)

4421

In [109]:
#Filtramos dataframe de las reviews de alabama
alabama[alabama['Id_Restaurant'].isin(ids_modificar)]

,Id_Usuario,Id_Restaurant,Rating,Reseña,Timestamp
939,1.011846e+20,0x8862b4c5990dab45:0x53865257e76430d4,1,"Certain people should not work there, working...",1628142224272
940,1.113374e+20,0x8862b4c5990dab45:0x53865257e76430d4,3,Depending on who is working and what time of t...,1485281653180
941,1.131096e+20,0x8862b4c5990dab45:0x53865257e76430d4,5,Never crowded fast service,1488085297698
942,1.080478e+20,0x8862b4c5990dab45:0x53865257e76430d4,3,Very had an issue with their gas but pumps are...,1474049284532
943,1.166074e+20,0x8862b4c5990dab45:0x53865257e76430d4,3,Gas pumps out dated,1486757915033
...,...,...,...,...,...
167482,1.060800e+20,0x88892443fb4c0b99:0x7e7164925cc33526,5,NaN,1517062790869
167483,1.089765e+20,0x88892443fb4c0b99:0x7e7164925cc33526,5,NaN,1517630940095
167484,1.111538e+20,0x88892443fb4c0b99:0x7e7164925cc33526,3,NaN,1517664457695
167485,1.114773e+20,0x88892443fb4c0b99:0x7e7164925cc33526,4,NaN,1514827344200


In [110]:
#Vemos cuantos únicos hay alabama
len(alabama[alabama['Id_Restaurant'].isin(ids_modificar)]['Id_Restaurant'].unique())

41

In [111]:
#Definimos la función para realizar el reemplazo de los ids de los usuarios
def cambioDeIndices (estado_df):
    for f in range(duplicados_google_df.shape[0]):
        reemplazador = duplicados_google_df['Conservar'][f]
        for c in duplicados_google_df['Cambiar'][f]:
            if estado_df[estado_df['Id_Restaurant']== c].index.size>0:
                for i in estado_df[estado_df['Id_Restaurant']== c].index:
                    estado_df.at[i,'Id_Restaurant'] = reemplazador
    return estado_df

Las siguientes celdas van a ejecutarse para comprobar que la función definida funcione correctamente.

In [112]:
#Los buscamos en el dataframe de duplicados
indices = []
for i in range(duplicados_google_df.shape[0]):
    for j in duplicados_google_df['Cambiar'][i]:
        if j in alabama[alabama['Id_Restaurant'].isin(ids_modificar)]['Id_Restaurant'].unique():
            indices.append(i)
            break
duplicados_google_df[duplicados_google_df.index.isin(indices)]

,Conservar,Cambiar
231,0x8862815add89f0d1:0x771245c4a67928e8,[0x88628479181a6849:0xe1f6b48c132b04b8]
912,0x888bb114bc1652a3:0x28d9ab3692a22d6b,[0x888bb1155d2e3203:0xf0ee35699c8a9f1]
923,0x889a4da3954692bb:0x81a5b54e0c529f20,"[0x889a4d9c88d21c09:0x2f5c81e8a0ba08c7, 0x889a..."
1031,0x88893d539cf5b181:0x7ef41d559f2f50a2,[0x809ade0743de662f:0xee1b0d79fc92c77]
1038,0x889a079590d2c57d:0x117d2a67160efbd4,[0x889a07946199476d:0x2d02a6eabc916aad]
1051,0x887d4f2e1859725b:0x5a712c49d4325a5c,[0x887d48d2068ae9a3:0x4fa1fb6aa103b5f7]
1330,0x889a4d8e87ac3a41:0x52944077dacc3286,[0x889a4c4a49a964cb:0x38f75033bf2595ac]
1341,0x8889230a00e54853:0xba2532ce30fe830,[0x88892443fb4c0b99:0x7e7164925cc33526]
1360,0x887d484f20a82e33:0x2208f26035a82205,"[0x887d49e211019965:0x4928fc5a0a54d9bd, 0x887d..."
1362,0x887d4e66718860af:0x9a48f95513e18cca,"[0x887d4f22b578c08b:0xbe18611cf7c881e1, 0x8862..."


In [113]:
#Vemos que sean menos de 41 indices los que aparecen acá
duplicados_google_df[duplicados_google_df.index.isin(indices)].shape[0]

36

In [115]:
#Vemos cuantos ids de restaurant únicos tiene el dataframe de Alabama
alabama['Id_Restaurant'].unique().size

1891

In [117]:
#Realizamos el cambio
alabama = cambioDeIndices(alabama)
alabama['Id_Restaurant'].unique().size

1856

Vemos que hay menos ids únicos. 35 menos. A los 41 ids que había lo reemplazaron 36. 35 de estos 36 ya estaban en Alabama. Pero 1 no. Entonces hay 35 ids menos.

In [118]:
#Vemos si se modificaron como quisimos
alabama.loc[939:943]

,Id_Usuario,Id_Restaurant,Rating,Reseña,Timestamp
939,1.011846e+20,0x887d4e66718860af:0x9a48f95513e18cca,1,"Certain people should not work there, working...",1628142224272
940,1.113374e+20,0x887d4e66718860af:0x9a48f95513e18cca,3,Depending on who is working and what time of t...,1485281653180
941,1.131096e+20,0x887d4e66718860af:0x9a48f95513e18cca,5,Never crowded fast service,1488085297698
942,1.080478e+20,0x887d4e66718860af:0x9a48f95513e18cca,3,Very had an issue with their gas but pumps are...,1474049284532
943,1.166074e+20,0x887d4e66718860af:0x9a48f95513e18cca,3,Gas pumps out dated,1486757915033


In [120]:
#Se modificó como quisimos. Reseteamos indice y vemos que no se haya perdido información
alabama.reset_index(inplace=True,drop=True)
alabama.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166894 entries, 0 to 166893
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Id_Usuario     166894 non-null  float64
 1   Id_Restaurant  166894 non-null  object 
 2   Rating         166894 non-null  int64  
 3   Reseña         93861 non-null   object 
 4   Timestamp      166894 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 6.4+ MB


In [134]:
#Extendemos todo el proceso para el resto de los estados
for e in estados:
    estado = pd.read_csv("reviews-estados/"+e+".csv", sep=";")                          #Importamos el dataframe
    estado.drop(columns=['name','pics','resp'],inplace=True)                            #Descartamos columnas que no se usan
    estado.rename(columns={'user_id':'Id_Usuario','time':'Timestamp',                   #Renombramos columnas
                            'text':'Reseña','gmap_id':'Id_Restaurant','rating':'Rating'},inplace=True)
    estado = estado[['Id_Usuario','Id_Restaurant','Rating','Reseña','Timestamp']]       #Reordenamos
    estado.drop_duplicates(inplace=True)                                                #Descartamos duplicados
    estado = cambioDeIndices(estado)                                                    #Cambiamos los indices
    estado.to_csv('reviews-estados/'+e+'.csv',index=False,sep=';',escapechar='\\')      #Exportamos a csv

Finalmente vamos a ver como nos quedó nuestro dataset. Cantidad de registros, usuarios, restaurants, etc.

In [135]:
#Concatenamos todo
cant_registros = 0
for e in estados:
    if e == 'Alabama':
        todos_estados = pd.read_csv('reviews-estados/'+e+'.csv',sep=';')
    else:
        todos_estados = pd.concat([todos_estados,pd.read_csv('reviews-estados/'+e+'.csv',sep=';')],ignore_index=True)

In [136]:
#Vemos cuantos registros quedaron
todos_estados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11422830 entries, 0 to 11422829
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Id_Usuario     float64
 1   Id_Restaurant  object 
 2   Rating         int64  
 3   Reseña         object 
 4   Timestamp      int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 435.7+ MB


In [137]:
#Cuantos usuarios
todos_estados['Id_Usuario'].unique().size

5629446

In [138]:
#Cuantos restaurants
todos_estados['Id_Restaurant'].unique().size

124606

Lo último que vamos a realizar es eliminar los restaurants que no vamos a utilizar porque no tenemos información en las reviews.

In [139]:
#Importamos el dataset
restaurants_google = pd.read_pickle('restaurants_google.pkl')
restaurants_google.head()

,Id_Restaurant,Nombre,Ciudad,Estado,Cod_postal,Latitud,Longitud,Tipo,Atributos
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,Los Angeles,CA,90005,34.058092,-118.292130,[Korean restaurant],"{'Service options': ['Takeout', 'Dine-in', 'De..."
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,La Mirada,CA,90638,33.916402,-118.010855,[Restaurant],"{'Service options': ['Outdoor seating', 'Curbs..."
2,0x87ec235c54d25b31:0x3b75fb5facc602f,"Sweet Rewards Gluten Free Bakery, LLC",Waukee,IA,50263,41.616079,-93.865487,"[Bakery, Health food restaurant]",{'Service options': ['Delivery']}
3,0x89c6c89efcaed69d:0xded973f6033e7dba,Vivi Bubble Tea,Philadelphia,PA,19147,39.940293,-75.150923,[Restaurant],"{'Service options': ['Delivery'], 'Amenities':..."
4,0x7c00456eecad3111:0x8217f9600c51f33,Hale Pops,Laie,HI,96762,21.637796,-157.920714,[Restaurant],"{'Service options': ['Outdoor seating', 'Takeo..."


In [140]:
restaurants_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209123 entries, 0 to 209122
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Id_Restaurant  209123 non-null  object 
 1   Nombre         209123 non-null  object 
 2   Ciudad         209123 non-null  object 
 3   Estado         209123 non-null  object 
 4   Cod_postal     209123 non-null  object 
 5   Latitud        209123 non-null  float64
 6   Longitud       209123 non-null  float64
 7   Tipo           209123 non-null  object 
 8   Atributos      207894 non-null  object 
dtypes: float64(2), object(7)
memory usage: 14.4+ MB


In [142]:
#Filtramos
restaurants_google_con_reviews = restaurants_google[restaurants_google['Id_Restaurant'].isin(todos_estados['Id_Restaurant'].unique())]
restaurants_google_con_reviews.reset_index(inplace=True,drop=True)
restaurants_google_con_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123911 entries, 0 to 123910
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Id_Restaurant  123911 non-null  object 
 1   Nombre         123911 non-null  object 
 2   Ciudad         123911 non-null  object 
 3   Estado         123911 non-null  object 
 4   Cod_postal     123911 non-null  object 
 5   Latitud        123911 non-null  float64
 6   Longitud       123911 non-null  float64
 7   Tipo           123911 non-null  object 
 8   Atributos      123450 non-null  object 
dtypes: float64(2), object(7)
memory usage: 8.5+ MB


Observamos que contamos con menos registros que los únicos de las reviews. Vamos a eliminar estos y finalmente tenemos los datasets listos.

In [145]:
#Filtramos el dataframe y reseteamos el indice
reviews = todos_estados[todos_estados['Id_Restaurant'].isin(restaurants_google_con_reviews['Id_Restaurant'].values)]
reviews.reset_index(inplace=True,drop=True)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11291393 entries, 0 to 11291392
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Id_Usuario     float64
 1   Id_Restaurant  object 
 2   Rating         int64  
 3   Reseña         object 
 4   Timestamp      int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 430.7+ MB


In [147]:
#Vemos cantidad de usuarios
reviews['Id_Usuario'].unique().size

5592606

In [149]:
#Vemos que coincidan las cantidades de restaurants
reviews['Id_Restaurant'].unique().size

123911

In [146]:
#Exportamos
reviews.to_csv('reviews-estados/reviews.csv',index=False,sep=';',escapechar='\\')
restaurants_google_con_reviews.to_pickle('metadata-sitios/restaurants_google.pkl')

Resumen:

* Cantidad de reviews: 11.291.393
* Cantidad de restaurants: 123.911
* Cantidad de usuarios: 5.592.606